# Manipulation.py: Pick and Place example

In [ ]:
import numpy as np
import robotic as ry
import manipulation as manip

In [ ]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))

midpoint = np.array([-0.105, 0.4, 0.705-.025+.1])
C.addFrame("box") \
    .setPosition(midpoint) \
    .setShape(ry.ST.ssBox, size=[0.08, 0.12, 0.08, 0.001]) \
    .setColor([1, 0, 0]) \
    .setContact(1) \
    .setMass(.1)

# For convenience, a few definitions:
gripper = "l_gripper"
box = "box"
table = "table"

C.view()

In [ ]:
def pick_place(obj: str, grasp_direction: str, place_direction: str, place_position: list[float], info: str="", vis: bool=False) -> bool:
	M = manip.ManipulationModelling(C, info, helpers=[gripper])
	M.setup_pick_and_place_waypoints(gripper, obj, homing_scale=1e-1)
	M.grasp_box(1., gripper, obj, "l_palm", grasp_direction)
	M.place_box(2., obj, table, "l_palm", place_direction)
	M.target_relative_xy_position(2., obj, table, place_position)
	M.solve()
	if not M.feasible:
		return False, []

	M1 = M.sub_motion(0)
	M1.keep_distance([.3,.7], "l_palm", obj, margin=.05)
	M1.retract([.0, .2], gripper)
	M1.approach([.8, 1.], gripper)
	path1 = M1.solve()
	if not M1.feasible:
		return False, []

	M2 = M.sub_motion(1)
	M2.keep_distance([], table, "panda_collCameraWrist")
	M2.keep_distance([.2, .8], table, obj, .04)
	M2.keep_distance([], "l_palm", obj)
	path2 = M2.solve()
	if not M2.feasible:
		return False, []

	if vis:
		M1.play(C, 1.)
		C.attach(gripper, obj)
		M2.play(C, 1.)
		C.attach(table, obj)

	path = np.append(path1, path2, 0)
	return True, path

In [ ]:
bot = ry.BotOp(C, useRealRobot=False)
bot.home(C)

In [ ]:
attempt_count = 30
success_count = 0

for l in range(attempt_count):

	target_position = [
		midpoint[0] + np.random.uniform(-.15, .15),
		midpoint[1] + np.random.uniform(-.15, .15),
		0.]
	
	success, path = pick_place(box, target_position, str(l), vis=True)

	if success:
		bot.move(path, [3])
		while bot.getTimeToEnd() > 0.:
			bot.sync(C)
		
	success_count += 1 if success else 0
	
print(f"Successful motions: {success_count}/{attempt_count}")